In [1]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

In [7]:
import cv2
import numpy as np

# Load the template and search image
img = cv2.imread('5258-601-A1-0414.jpeg', 1)  
template = cv2.imread('template (6).png', 1) 

sift = cv2.xfeatures2d.SIFT_create()


In [9]:
kp1, des1 = sift.detectAndCompute(img,None)
kp2, des2 = sift.detectAndCompute(template,None)

In [10]:
# BF(brute force)Matcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

In [11]:

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
     

In [ ]:
#img3 = cv2.drawMatchesKnn(img,kp1,template,kp2,good,None,flags=2)
#cv2.imwrite("shift match.jpeg", img3)

In [12]:
def merge_rectangles(rectangles):
    merged_rectangles = []
    for rectangle in rectangles:
        x, y, w, h = rectangle
        merged = False
        for i, merged_rectangle in enumerate(merged_rectangles):
            mx, my, mw, mh = merged_rectangle
            if x >= mx and y >= my and x + w <= mx + mw and y + h <= my + mh:
                # Rectangle is completely inside an existing merged rectangle
                merged_rectangles[i] = (mx, my, mw, mh)
                merged = True
                break
            elif x <= mx and y <= my and x + w >= mx + mw and y + h >= my + mh:
                # Rectangle completely covers an existing merged rectangle
                merged_rectangles[i] = (x, y, w, h)
                merged = True
                break
            elif x < mx + mw and y < my + mh and x + w > mx and y + h > my:
                # Rectangles overlap, create a new merged rectangle
                merged_rectangles[i] = (
                    min(x, mx),
                    min(y, my),
                    max(x + w, mx + mw) - min(x, mx),
                    max(y + h, my + mh) - min(y, my),
                )
                merged = True
                break
        if not merged:
            # No overlap, add a new merged rectangle
            merged_rectangles.append((x, y, w, h))
    return merged_rectangles


In [13]:
rois = []
for match in good:
    # Get the index of the keypoints in the first image
    query_idx = match.queryIdx
    keypoint = kp1[query_idx]

    # Define a ROI around the keypoint
    roi = (int(keypoint.pt[0]), int(keypoint.pt[1]), 100, 100)  # Adjust the size as needed

    # Add the ROI to the list
    rois.append(roi)
    

merge_roi = merge_rectangles(rois)    
for roi in merge_roi:
    # Draw the ROI on the first image
    cv2.rectangle(img, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (0, 255, 0), 2)


In [14]:
cv2.imwrite("roi_shift_match_4.jpeg", img)

True